In [1]:
import urllib.request
import pandas as pd
import requests


In [19]:
confirmation_url = "https://www.microsoft.com/en-us/download/confirmation.aspx?id=56519"
#confirmation_url = "https://www.microsoft.com/en-us/download/confirmation.aspx?id=56519"

download_url = "https://www.microsoft.com/en-us/download/details.aspx?id=56519"
# https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20240930.json

import json
import requests
from bs4 import BeautifulSoup
import time


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


# Making a GET request
r = requests.get(confirmation_url, headers=headers)
 
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')




In [ ]:
url=""
#url = "https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220404.json"
# find all the anchor tags with "href"
for link in soup.find_all('a'):
    if "ServiceTags_Public" in link.get('href'):
        url=link.get('href')
        break

print(url)


In [23]:

data = json.loads(requests.get(url).text)

df_tags = pd.json_normalize(data, record_path=['values'], meta=['changeNumber','cloud'],errors='ignore')

## Get IP Ranges filtered for Power BI

Pass the IP ranges to a web service to do reverse DNS lookup

In [ ]:
#df_tags['properties.systemService'].unique()
#df_tags[df_tags['properties.systemService']=="AzureDatabricks"]


list(df_tags[df_tags['properties.systemService']=="PowerBI"]['properties.addressPrefixes'])[0]

In [38]:
df = pd.DataFrame()
import json
import os
import ipaddress

from dotenv import dotenv_values

datacenters = []

# settings = dotenv_values(".env")

PACKAGE="WS5"
# API_KEY=settings['API_KEY']

API_KEY=get_from_kv("ip2location-api-key")

# Sample list of IP addresses
ipranges = list(df_tags[df_tags['properties.systemService']=="PowerBI"]['properties.addressPrefixes'])[0]

# Extracting IPv4 addresses
ipv4_addresses = [ip for ip in ipranges if isinstance(ipaddress.ip_network(ip), ipaddress.IPv4Network)]

#https://api.ip2location.io/?key=&ip=8.8.8.8&format=json    

# don't need to write to a local file anymore
#with open('iprange_doc.json', 'w') as f:
for ip in ipv4_addresses:
    request = f"https://api.ip2location.io/?key={API_KEY}&ip={ip.split('/')[0]}&format=json"    
    #request = f"https://api.ip2location.com/v2/?ip={ip.split('/')[0]}&key={API_KEY}&package={PACKAGE}"
    my_request = request
    response = requests.get(my_request)
    df_temp = response.json()
    df_temp['CIDR']=ip
    datacenters.append(df_temp)

In [39]:
with open('datacenters.json', 'w') as f:
    json.dump(datacenters, f)


In [ ]:
df = pd.json_normalize(datacenters)
df.head(20)

In [43]:
df.to_excel("/mnt/c/Projects/Fabric/ipranges/data/powerbi_ips.xlsx")


In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode_outer

# Initialize Spark session
spark = SparkSession.builder.appName("ReadAzureRegions").getOrCreate()

# Read the JSON file
# df_azure_regions = spark.read.json("azure_regions.json")

df = spark.read.option("multiline", "true").json("azure_regions.json")

# Show the dataframe
# df_azure_regions.show()

In [ ]:

df.printSchema()

In [ ]:
dd = df.selectExpr(
    "displayName",
    "id",
    "metadata.geography as geography",
    "metadata.geographyGroup as geographyGroup",
    "metadata.latitude as latitude",
    "metadata.longitude as longitude",
    "metadata.physicalLocation as physicalLocation",
    "metadata.regionCategory as regionCategory",
    "metadata.regionType as regionType",
    "regionalDisplayName",
    "type",
    "name",
    "explode_outer(metadata.pairedRegion) as pairedRegion",
    "availabilityZoneMappings")

# dd.show()
de = dd.selectExpr(
    "*",
    "pairedRegion.id as pairedRegionId",
    "pairedRegion.name as pairedRegionName").drop("pairedRegion")

de.show()

In [ ]:
de.printSchema()

In [ ]:
dg = de.withColumn("zones", explode_outer("availabilityZoneMappings")).drop("availabilityZoneMappings")
dh = dg.selectExpr(
    "*",
    "zones.logicalZone as logicalZone",
    "zones.physicalZone as physicalZone").drop("zones")
dg.limit(10).show()

In [ ]:
dh.limit(10).show()

dh.write.mode("overwrite").parquet("azure_regions.parquet")
dh.toPandas().to_excel("azure_regions.xlsx")

In [10]:
import pandas as pd

df = pd.read_excel("powerbi_ips.xlsx")

In [ ]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.identity import ClientSecretCredential
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient
from datetime import datetime, timedelta
import time

from azure.mgmt.storage.models import (
    StorageAccountCreateParameters,
    NetworkRuleSet,
    IPRule,
    StorageAccountUpdateParameters,
    Sku,
    SkuName,
    Kind
)

In [ ]:
import os
subscription_id = os.getenv("SUBSCRIPTION_ID")

credentials = ServicePrincipalCredentials(
    client_id = os.getenv("CLIENT_ID"),
    secret=os.getenv("site_ADF_SP_SECRET"),
    tenant=os.getenv("TENANT_ID")
)

blob_credentials = ClientSecretCredential(
    client_id = 'a888b9fe-38ff-4551-844f-7416e1cbb89f',
    client_secret=os.getenv("site_ADF_SP_SECRET"),
    tenant_id=os.getenv("TENANT_ID")
)

GROUP_NAME = 'site-rg'
STORAGE_ACCOUNT_NAME = 'sitegen2'

#credentials = DefaultAzureCredential(exclude_interactive_browser_credential=False)

resource_client = ResourceManagementClient(credentials, subscription_id)
storage_client = StorageManagementClient(credentials, subscription_id)

In [ ]:
storage_account = storage_client.storage_accounts.get_properties("site-rg","sitegen2")

In [ ]:
ip = df[df['region_name']=='Iowa']['CIDR']

str_ip_rules = ""

ip_ranges2 = []

# /31 and /32 are not allowed to be entered as IP Ranges using CIDR
# must be entered as an IP
for address in ip:
    if ":" not in address:
        if "31" in address:
            val = address.replace("/31","")
            str_ip_rules += f"IPRule(ip_address_or_range='{val}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=val, action='Allow'))
            val = val.rsplit(".",1)
            val = val[0] + "." + str(int(val[1])+1)
            str_ip_rules += f"IPRule(ip_address_or_range='{val}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=val, action='Allow'))


        else:
            str_ip_rules += f"IPRule(ip_address_or_range='{address}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=address, action='Allow'))


In [ ]:
str_ip_rules=str_ip_rules[0:-1]
ip_rules = list(str_ip_rules.split("|"))

### Working Example

This shows what it looks like if just entering by hand

In [ ]:
ips=['20.37.156.200/30','20.37.156.240/28']

storage_account = storage_client.storage_accounts.update(
    GROUP_NAME, STORAGE_ACCOUNT_NAME,
    StorageAccountUpdateParameters (
        network_rule_set = NetworkRuleSet(
            default_action='Deny',
            ip_rules = [
                  IPRule(ip_address_or_range='20.37.156.200/30', action='Allow')
                , IPRule(ip_address_or_range='20.37.156.240/28', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.124', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.125', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.208/30', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.216/29', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.224/28', action='Allow')
                , IPRule(ip_address_or_range='20.45.242.48/29', action='Allow')
                , IPRule(ip_address_or_range='20.46.15.56/30', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.48/28', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.64/30', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.112/29', action='Allow')
                , IPRule(ip_address_or_range='20.98.146.0/27', action='Allow')
            ]
        )
    )
)

### This works

This is how to add the list of IPRules through dynamic coding

In [ ]:
storage_account = storage_client.storage_accounts.update(
    GROUP_NAME, STORAGE_ACCOUNT_NAME,
    StorageAccountUpdateParameters (
        network_rule_set = NetworkRuleSet(
            default_action='Deny',
            ip_rules = ip_ranges2
        )
    )
)

```
StorageAccountUpdateParameters(*,  
**sku:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Sku] = None,  
**tags:** Optional[Dict[str, str]] = None, identity: Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Identity] = None,   
**kind:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.Kind]] = None, 
**custom_domain:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.CustomDomain] = None,  
**encryption:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Encryption] = None,  
**access_tier:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.AccessTier]] = None,  **azure_files_identity_based_authentication:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.AzureFilesIdentityBasedAuthentication] = None,   
**enable_https_traffic_only:** Optional[bool] = None,  
**network_rule_set:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.NetworkRuleSet] = None,  
**large_file_shares_state:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.LargeFileSharesState]] =None,  **allow_blob_public_access:** Optional[bool] = None,  
**minimum_tls_version:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.MinimumTlsVersion]] = None,  **allow_shared_key_access:** Optional[bool] = None, **kwargs)
```

```
NetworkRuleSet(*,  
*default_action:* Union[str, azure.mgmt.storage.v2021_01_01.models._storage_management_client_enums.DefaultAction] = 'Allow',  
*bypass:* Optional[Union[str, azure.mgmt.storage.v2021_01_01.models._storage_management_client_enums.Bypass]] = 'AzureServices',  
*resource_access_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.ResourceAccessRule]] = None,  
*virtual_network_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.VirtualNetworkRule]] = None,   
*ip_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.IPRule]] = None, **kwargs)
```

In [ ]:
from azure.storage.blob import BlobServiceClient

service = BlobServiceClient(account_url="https://sitegen2.blob.core.windows.net/", credential=blob_credentials)
#service.create_container("testing1")